# Video Search Engine

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import speech_recognition as sr
from math import floor
from moviepy.editor import VideoFileClip
import os

class VideoSearchApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Search Engine")

        # Increase the font size
        self.font_size = ('Helvetica', 14)

        # Create GUI elements
        self.heading_label = tk.Label(root, text="Video Search Engine", font=('Helvetica', 18, 'bold'))
        self.heading_label.pack(pady=10)

        self.label_file = tk.Label(root, text="Enter video file or use Browse:", font=self.font_size)
        self.label_file.pack(pady=10)

        self.entry_file = tk.Entry(root, width=50, font=self.font_size)
        self.entry_file.pack(pady=10)

        self.browse_button = tk.Button(root, text="Browse", command=self.browse_file, font=self.font_size)
        self.browse_button.pack(pady=10)

        self.label_input = tk.Label(root, text="Enter text for video search:", font=self.font_size)
        self.label_input.pack(pady=10)

        self.entry_input = tk.Entry(root, width=50, font=self.font_size)
        self.entry_input.pack(pady=10)

        self.search_button = tk.Button(root, text="Search", command=self.search_video, font=self.font_size)
        self.search_button.pack(pady=10)
        
        # Header for search results
        self.heading_label = tk.Label(root, text="Search Results", font=('Helvetica', 16, 'bold'))
        self.heading_label.pack(pady=10)

        self.results_text = scrolledtext.ScrolledText(root, width=80, height=10, font=self.font_size)
        self.results_text.pack(pady=10)

    def browse_file(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.entry_file.delete(0, tk.END)
        self.entry_file.insert(0, self.file_path)

    def search_video(self):
        if not self.file_path:
            messagebox.showinfo("Error", "Please select a video file.")
            return

        self.user_input = self.entry_input.get()
        if not self.user_input:
            messagebox.showinfo("Error", "Please enter a search query.")
            return

        try:
            # Extract audio from the video using moviepy
            video_file_path = self.file_path
            video = VideoFileClip(video_file_path)
            audio_file_path = tempfile.NamedTemporaryFile(suffix=".wav").name
            video.audio.write_audiofile(audio_file_path, codec='pcm_s16le')

            # Use the Google Web Speech API for speech recognition
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio_file_path) as audio_file:
                audio_data = recognizer.record(audio_file)

            # Get timestamps and words from the speech recognition results
            results = self.get_search_results(audio_data, video)

            # Display results on the interface
            self.display_results(results)

        except sr.UnknownValueError:
            messagebox.showinfo("Error", "Speech recognition could not understand the audio.")
        except sr.RequestError as e:
            messagebox.showinfo("Error", f"Could not request results from Google Speech Recognition service; {e}")
        except FileNotFoundError as e:
            messagebox.showinfo("Error", f"File not found error: {e}")
        except Exception as e:
            messagebox.showinfo("Error", f"An unexpected error occurred: {e}")
        finally:
            # Clean up: Remove the temporary audio file
            if os.path.exists(audio_file_path):
                os.remove(audio_file_path)

    def get_search_results(self, audio_data, video):
        recognizer = sr.Recognizer()
        response = recognizer.recognize_google(audio_data, show_all=True)
        words_info = response.get('alternative', [])[0].get('transcript', '').split()
        search_results = []
        if words_info:
            current_time = 0.0
            for word in words_info:
                word_duration = video.duration / len(words_info)
                if self.user_input.lower() in word.lower():
                    if current_time>10:
                        search_results.append((self.user_input, f"{current_time-10:.2f} seconds"))
                    else:
                        search_results.append((self.user_input, f"{floor(current_time):.2f} seconds"))

                current_time += word_duration

        return search_results

    def display_results(self, results):
        self.results_text.delete(1.0, tk.END)  # Clear previous results
        if not results:
            self.results_text.insert(tk.END, "No matching results found.")
        else:
            for i, (query, timestamp) in enumerate(results, start=1):
                result_text = f".{query} found at {timestamp}\n"
                self.results_text.insert(tk.END, result_text)
                self.results_text.tag_add(f"tag_{i}", f"{i}.0", f"{i}.end")
                self.results_text.tag_configure(f"tag_{i}", foreground="blue", underline=True)
                self.results_text.tag_bind(f"tag_{i}", "<Button-1>", lambda event, t=timestamp: self.play_video(t))

            self.results_text.insert(tk.END, "\nClick on timestamps to view the video from that time.")
    def play_video(self, timestamp):
        try:
            if not os.path.exists(self.file_path):
                messagebox.showinfo("Error", "Video file not found.")
                return

            # Calculate the video start time in seconds
            target_time = self.get_total_seconds(timestamp)
            if target_time>10:
                target_time-=10
            else:
                target_time=floor(target_time)
            # Create a VideoFileClip starting from the target time
            video = VideoFileClip(self.file_path).subclip(target_time)

            # Play the video clip
            video.preview()
        finally:
            # Close the video clip to free up resources
            video.close()

    def get_total_seconds(self, timestamp):
        try:
            return float(timestamp.replace(" seconds", ""))
        except ValueError:
            messagebox.showinfo("Error", f"Invalid timestamp format: {timestamp}")
            return 0.0

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoSearchApp(root)
    root.mainloop()

MoviePy - Writing audio in C:\Users\User\AppData\Local\Temp\tmpjknnyck_.wav


MoviePy - Done.
